# Program Overview
<h4>Purpose:</h4> 
The purpose of this program is to leverage the LegiScan API to automate the collection, identification, and extraction of bill data, minizing manual efforts and improving efficiency.  

<br><h4>Requirements:</h4>
To run this program you'll need the following
- API Key from https://legiscan.com/
- Permission to Download & Create new files
- Latest Version of Python Installed https://www.python.org/downloads/
- Reliable internet

<br><h4>Recommendations:</h4>
- It is recommended to run the program once per week to check for any updates to bill data
- To limit unnecessary API calls when monitoring a small number of states, adjust the *stateCodes* parameter as needed
- If the search criteria is too broad or too restrictive, refine or broaden the *relevanceFilter* and *searchTerms* parameters as needed
- Do not set relevanceFilter too low, as it'll result in too many subsequent API calls when collecting Bill Info

<br><h4>Features:</h4> 
- Minized API usage by downloading new datasets only when updated are detected
- Automatically updating bill data when changes are detected
- Adjustable filtering criteria through *relevanceFilter* and *searchTerms* parameters
- Aggregation of all collected bill data into a singular dataset for easy exporting and extraction
- Exporting all bill data into a single Excel workbook sorted by bill relevance

<br><h4>Outputs:</h4> 
The final codeblock in this program outputs a single Excel workbook with individual sheets that correspond to the dates in the *stateCodes* parameter.
Below is a breakdown of each of the columns, please note if a specific cell is empty, it's due to the API not providing an input for that given cell.
- *latest_update*: datetime of when the row was aggregated from the dataset, format: *MM-DD-YYYY* 
- *relevance*: a percentage from (0-100) representing how closely the bill matches the search criteria
- *state*: abbreviation of the state the bill is from
- *bill_number*: number of the bill
- *date_introduced*: date the bill was introduced
- *description*: description of the bill
- *status*: current status of the bill
    - Possible values include: *Introduced*, *Engrossed*, *Enrolled*, *Passed*, *Vetoed*, *Failed*, *N/A*
- *link*: direct link to the bill
- *chamber*: Legislative chamber associated with the bill
    - *Note*: *chamber* is inferred from the prefix of the *bill_number* and may be *House*, *Senate*, or the prefix itself
- *party*: political party of the primary sponsor
- *primary* sponsor: list of the primary sponsor
- *co-sponsor*: list of co-sponsors
- *joint sponsor*: list of joint sponsors
- *generic/unspecified sponsor*: list of generic/unspecified sponsors

# Config
This section will explain the variables in the config, the type of variables, and the formatting necessary for future edits. When making any edits to any existing variables, run the code block to ensure the changes have been saved. Failure to run the code block after making changes will lead to undesired outputs in the final Excel sheet, or the possibility of failing API calls due to an invalid API Key.

##### Variables
- apiKey [String]
    - This is the API Key you received from LegiScan creating an account.
    - When entering the key into the config, ensure that it is surrounded by quotation marks.
    - EX: apiKey = "YOUR_API_KEY_HERE"
- relevanceFilter [Integer]
    - The relevancy score is a percentage that represents how accurately a bill matches to your search terms.
    - By default, the filter percentage is preset to 20, therefore all bills are included. You can change this percentage from 0-100 based off how limited you want your search to be.
    - A "relevance" column is included in the final Excel Workbook, therefore you can leave this at 0 and use your own judgement to determine a bills relevancy.
    - If you do choose to modify *relevanceFilter*, enter it in with no quotation marks. The percentage you choose is the minimum required to be included in the final Excel workbook.
    - EX: relevanceFilter = 25 
- stateCodes [List]
    - These are the abbreviations for all 50 states, plus the Federal Government which is represented as "DC".
    - By default, all 50 states plus DC have been included. If you want limit your search, simply remove "[STATE ABBREVIATION]", from the *stateCodes* variable.
    - If you're only looking to select a few states, it's best practice to remove any additional ones you don't need to save on API calls.
    - EX: stateCodes = ["DC","MD","VA"] --> stateCodes = ["DC","VA"]
- searchTerms [List]
    - These are the search terms/criteria the program will search for across all the bills in the latest session for all of the states in *stateCodes*.
    - To broaden your search, simply add a new line and enter in additional search terms following the existing format, and vice versa to limit your search further.
    - EX: searchTerms = ["death penalty","execution method"] --> searchTerms = ["death penalty", "execution method", "capital murder"]

In [0]:
# Abeezar's API Key (30,000/mo limit)
apiKey = "e6cc3653ff0b22a524c6062132a65fd9"

relevanceFilter = 1
stateCodes = [
    "US", "DC", "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", 
    "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", 
    "ME", "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", 
    "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", 
    "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", 
    "WI", "WY"
]
searchTerms = [
    "death penalty",
    "capital punishment",
    "capital sexual battery",
    "execution method",
    "death sentence",
    "method of execution"
    ]


# Initializing Program
This section imports any required libraries to run the program itself, as well as create the directories, files, and JSON structures. After running the Config codeblock you must run the codeblock below, otherwise subsequent codeblocks will be unable to run due to un-imported dependencies. This codeblock will also print out any potential errors when creating the directories and files. Should there be any errors, resolve them before moving forward to prevent unnecessary API calls.

When a directory or file already exists, you will get the following output. "*Error: The directory for "[STATE]" already exists.*" This can be ignored, as the files were successful in creation in a prior run of the codeblock.

In [0]:
# Import statements
import requests
import pandas as pd
import os
import zipfile
import shutil
import json
import datetime
import re
import openpyxl


# Creating stateData folder to store raw and aggregated data
try:
    os.makedirs("stateData")
except PermissionError:
    print("Permission Denied: Unable to create directory")
except OSError as e:
    if e.errno == 17:
        print("The directory for \"stateData\" already exists.")
    else:
        print(e)
except Exception as e:
    print("[ERROR]: " + str(e))

# Creating folders for each state inside stateData
for state in stateCodes:
    try:
        os.makedirs("stateData/" + state)
        os.makedirs("stateData/" + state + "/data")
        os.makedirs("stateData/" + state + "/dataset_" + state)
    except PermissionError:
        print("Permission Denied: Unable to create directory: ")
    except OSError as e:
        if e.errno == 17:
            print("The directory for \"" + state + "\" already exists.")
        else:
            print(e)
    except Exception as e:
        print("[ERROR]: " + str(e))

    # Checking to see if dataset_info exists, creating if doesn't
    if not os.path.exists("stateData/" + state + "/data/dataset.json"):
        
        # JSON Structure for each state's dataset    
        data = {
            "date": "",
            "session_id": "",
            "dataset_hash": "",
            "access_key": ""
        }
        
        # Saving JSON structure locally, to edit later
        with open("stateData/" + state + "/data/dataset.json", "w") as file:
            json.dump(data, file, indent=5)


# Downloading Datasets & Locally Storing
The codeblock below serves the purpose of finding the dataset for the latest session, checking to see if any updates were made, and downloading the new dataset if updates were made. Run this codeblock on a weekly basis to update your local datasets. Should there be any errors when pulling data from the LegiScan API, an error message with the *status_code* will be outputted to help identify and diagnose the error.

Only run this section once per week, if you're doing multiple searchers per week then **DO NOT** run this section, skip over it.

In [0]:
# Getting dataset locally and extracting into stateData directory for each state
for state in stateCodes:
    
    # LegiScan API call for each state's Datasets
    searchURL = "https://api.legiscan.com/?key=" + apiKey + "&op=getDatasetList&state=" + state
    response = requests.get(searchURL)
    
    # Checking to see if API call was successful
    if response.status_code == 200:
        latestSession = response.json()["datasetlist"][0]

        # Reading State's Dataset Info
        with open("stateData/" + state + "/data/dataset.json", "r") as file:
            saved_data = json.load(file)

        # Checking to see if Local Dataset Hash matches API's Dataset Hash
        # If Local does match API, we skip over downloading a new Dataset
        # If Local doesn't match API, we'll download a new Dataset
        if str(saved_data["dataset_hash"]) == str(latestSession["dataset_hash"]):
            print("No update needed for " + state +  "'s Dataset. Local dataset_hash matches the API. Only updating \"Date\" in dataset.json")
            saved_data["date"] = str(datetime.datetime.now())
            with open("stateData/" + state + "/data/dataset.json", "w") as file:
                json.dump(saved_data, file, indent=5)

            continue
        else:
            # Local and API dataset hashes don't match, so we'll download a new dataset
            
            # Updating dataset.json with updated information from API
            saved_data["date"] = str(datetime.datetime.now())
            saved_data["session_id"] = str(latestSession["session_id"])
            saved_data["dataset_hash"] = str(latestSession["dataset_hash"])
            saved_data["access_key"] = str(latestSession["access_key"])

            # Saving dataset.json file in the stateData directory with updated dataset info
            with open("stateData/" + state + "/data/dataset.json", "w") as file:
                json.dump(saved_data, file, indent=5)

            # LegiScan API call to download state's dataset
            searchURL = searchURL = "https://api.legiscan.com/?key=" + apiKey + "&op=getDatasetRaw&access_key=" + latestSession["access_key"] + "&id=" + str(latestSession["session_id"]) + "&format=csv"
            response = requests.get(searchURL)
            
            # Checking to see if API call was successful
            if response.status_code == 200:

                # Storing dataset in state's folder
                with open("stateData/" + state + "/dataset_Latest.zip", "wb") as file:
                    for chunk in response.iter_content(chunk_size=8192):
                        file.write(chunk)
                    print("Finished Downloading Dataset for: " + state)
                
                # Extracting datasets from downloaded Zip file
                with zipfile.ZipFile("stateData/" + state +"/dataset_Latest.zip" , "r") as zipFolder:
                    zipFolder.extractall(path="stateData/")

                # Finding the name of the dataset folder we extracted 
                for folder in os.listdir("stateData/" + state):
                    if folder != "dataset_Latest.zip" and folder != "data" and not folder.startswith("dataset_"):
                        session_info = folder

                # Moving dataset CSV files to dataset_STATE folder
                for file in os.listdir("stateData/" + state + "/" + session_info + "/csv"):
                    source = os.path.join("stateData/" + state + "/" + session_info + "/csv", file)
                    destination = os.path.join("stateData/" + state + "/dataset_" + state, file)
                    try:
                        shutil.move(source, destination)
                    except Exception as e:
                        print(f"Error moving file {file}: {str(e)}")
            else:
                print("LegiScan API call failed to download dataset.")
                print("State: " + state + " | Response Code: " + response.status_code)
                continue

    else:
        print("LegiScan API call failed to retrieve State's Dataset list")
        print("State: " + state + " | Response Code: " + response.status_code)
        continue


# Bill Search
The codeblock below will utilize LegiScan's API to search for bills within every state's latest session and store the search results locally to be aggregated later. Should there be any errors when pulling data from the LegiScan API, an error message with the *status_code* will be outputted to help identify and diagnose the error.

*Note: This section is very heavy on API calls, please avoid running it more than once per week unless necessary.*

In [0]:
# After downloading state's dataset locally, we'll use LegiScan's getSearchRaw to find bills that match our search criteria
for state in stateCodes:

    # Get the state's current session_id from dataset.json
    with open("stateData/" + state + "/data/dataset.json", "r") as file:
        saved_data = json.load(file)
        session_id = saved_data["session_id"]
    
    # Creating searchQuery from search criteria
    searchQuery = "(" + "+OR+".join(f'"{term}"' for term in searchTerms) + ")"
    
    # LegiScan API call to find matching bills
    searchURL = "https://api.legiscan.com/?key=" + apiKey + "&op=getSearchRaw&id=" + session_id + "&query=" + searchQuery
    response = requests.get(searchURL)
    
    # Checking to see if API call was successful
    if response.status_code == 200:
        # Saving response locally to aggregate data later
        with open("stateData/" + state + "/data/rawSearch.json", "w") as file:
            json.dump(response.json(), file, indent=5)
    else:
        print("LegiScan API call failed to retrieve search results for Bills")
        print("State: " + state + " | Response Code: " + response.status_code)


    # After saving the rawSearch locally, we're going to use the locally saved file to ensure it passes through the relevancyFilter
    # if it passes through the relevancy filter we'll save it in bills.json
    bills = {}

    with open("stateData/" + state + "/data/rawSearch.json","r") as file:
        rawSearch = json.load(file)

    # Skipping over bills if no search results, we want to create bills.json
    if "results" in rawSearch["searchresult"]: 
        for bill in rawSearch["searchresult"]["results"]:
            
            # if bill doesn't meet relevance filter, skip over it 
            if int(bill["relevance"]) < int(relevanceFilter): continue


            # use getBill to get raw bill data and save it into bills.json
            searchURL = "https://api.legiscan.com/?key=" + apiKey + "&op=getBill&id=" + str(bill["bill_id"])
            response = requests.get(searchURL)

            # Checking to see if the API call was successful
            if response.status_code == 200:
                rawBill = response.json()
                bills[bill["bill_id"]] = {
                    "agg": {
                        "latest_update": str(datetime.datetime.now().strftime("%m-%d-%Y")),
                        "change_hash": bill["change_hash"],
                        "relevance": bill["relevance"]
                    },
                    "raw": rawBill["bill"]
                }
            else:
                print("LegiScan API call failed to retrieve bill data.")
                print("State: " + state + " | Bill ID: " + bill["bill_id"] + " |Response Code: " + response.status_code)
            
    # Saving all relevant bills locally before aggregating by bill
    with open("stateData/" + state + "/data/bills.json", "w") as file:
        json.dump(bills, file, indent=5)
        print(state, 'dumping into JSON')


# Parsing & Aggregating Data
The codeblock below will parse through the search results from the LegiScan API, and aggregate each dataset into a singular JSON file to later be exported into the Excel Workbook.

In [0]:
# Dict for bill status'
statusDict = {
    0: "N/A", 1: "Introduced", 2: "Engrossed",
    3: "Enrolled", 4: "Passed", 5: "Vetoed",
    6: "Failed", 7: "Override", 8: "Chaptered",
    9: "Refer", 10: "Report Pass", 11: "Report DNP",
    12: "Draft"
}

# Dict for sponsor types
sponsorTypeDict = {
    0: "generic-unspecified", 1: "primary",
    2: "co", 3: "joint"
}

# Dict for sponsor parties
sponsorPartyDict = {
    "1": "democrat", "2": "republican", "3": "independent",
    "4": "green party", "5": "libertarian", "6": "nonpartisan"
}

# Parsing through and saving LegiScan getRawSearch pull to find relevant bills plus info to allow for data aggregation by bill
for state in stateCodes:

    # Reading in State's relevant dataset and aggregating by bill 
    billsCSV = pd.read_csv("stateData/" + state + "/dataset_" + state + "/bills.csv", on_bad_lines="skip")
    sponsorsCSV = pd.read_csv("stateData/" + state + "/dataset_" + state + "/sponsors.csv", on_bad_lines="skip")
    peopleCSV = pd.read_csv("stateData/" + state + "/dataset_" + state + "/people.csv", on_bad_lines="skip")
    historyCSV = pd.read_csv("stateData/" + state + "/dataset_" + state + "/history.csv", on_bad_lines="skip")

    # Merging together, bills, sponsors, people
    merged_df = pd.merge(billsCSV, historyCSV[["bill_id","chamber"]], on="bill_id", how="left")
    merged_df = pd.merge(merged_df, sponsorsCSV[["bill_id","people_id","position"]], on="bill_id", how="left")
    merged_df = pd.merge(merged_df, peopleCSV[["people_id","name","party"]], on="people_id", how="left")
    
    # Dropping duplicates and setting index
    merged_df.drop_duplicates(inplace=True)
    merged_df.set_index("bill_id", inplace=True) 

    # Read in bills.json to aggregate data by bill
    with open("stateData/" + state + "/data/bills.json", "r") as file:
        bills_json = json.load(file)

    # Aggregating data by bill
    for bill_id in bills_json:
        # Error Handling: If some reason we are unable to find the bill locally, skip over it
        if billsCSV[billsCSV["bill_id"] == int(bill_id)].empty: continue

        # Get the row in billsCSV that matches the bill_id in bills_json
        bill_info = billsCSV[billsCSV["bill_id"] == int(bill_id)]

        # Populating bills.json 
        bills_json[bill_id]["agg"]["state"] = state
        bills_json[bill_id]["agg"]["bill number"] = bill_info["bill_number"].values[0]
        bills_json[bill_id]["agg"]["description"] = bill_info["description"].values[0]
        bills_json[bill_id]["agg"]["state link"] = bill_info["state_link"].values[0]
        bills_json[bill_id]["agg"]["legiscan link"] = bill_info["url"].values[0]

        # Error Handling: Some bills have empty progress 
        if bills_json[bill_id]["raw"]["progress"]:
            bills_json[bill_id]["agg"]["status"] = {
                "date": datetime.datetime.strptime(bills_json[bill_id]["raw"]["progress"][-1]["date"], "%Y-%m-%d").strftime("%m-%d-%Y"),
                "label": statusDict.get(bills_json[bill_id]["raw"]["progress"][-1]["event"], " "),
                "action": bill_info["last_action"].values[0]
            }
            bills_json[bill_id]["agg"]["date introduced"] = datetime.datetime.strptime(bills_json[bill_id]["raw"]["progress"][0]["date"], "%Y-%m-%d").strftime("%m-%d-%Y")
        else:
            bills_json[bill_id]["agg"]["status"] = {
                "date": " ",
                "label": " ",
                "action": bill_info["last_action"].values[0]
            }
            bills_json[bill_id]["agg"]["date introduced"] = " "

        # Error Handling: Some bills provide party in a Series, some as a String
        party = merged_df.loc[int(bill_id)]["party"]
        if isinstance(party, pd.Series):
            bills_json[bill_id]["agg"]["party"] = merged_df.loc[int(bill_id)]["party"].values[0]
        else:
            bills_json[bill_id]["agg"]["party"] = merged_df.loc[int(bill_id)]["party"]

        # Populating chamber, if chamber is not H, A, S, or L, then prefix of bill
        if bills_json[bill_id]["raw"]["current_body"].startswith("H") or bills_json[bill_id]["raw"]["current_body"].startswith("A"):
            bills_json[bill_id]["agg"]["chamber"] = "House"
        elif bills_json[bill_id]["raw"]["current_body"].startswith("S"):
            bills_json[bill_id]["agg"]["chamber"] = "Senate"
        elif bills_json[bill_id]["raw"]["current_body"].startswith("L"):
            bills_json[bill_id]["agg"]["chamber"] = "Legislature"
        else:
            # Bills that don't start with H or S will use the prefix of the bill as the chamber
            billPrefix = re.match(r"^[A-Za-z]+", bill_info["bill_number"].values[0]).group(0)
            bills_json[bill_id]["agg"]["chamber"] = billPrefix

        # Creating json structure for sponsors
        bills_json[bill_id]["agg"]["sponsors"] = {
            "type": {
                "generic-unspecified": [],
                "primary": [],
                "co": [],
                "joint": []
            },
            "sponsor-party": {
                "democrat": 0,
                "republican": 0,
                "independent": 0,
                "green party": 0,
                "libertarian": 0,
                "nonpartisan": 0
            }
        }

        # Populating sponsor names and parties by type
        for sponsor in bills_json[bill_id]["raw"]["sponsors"]:
            
            # Pulling sponsor type from dict
            sponsor_type = sponsorTypeDict.get(sponsor["sponsor_type_id"])
            if sponsor_type is not None:
                bills_json[bill_id]["agg"]["sponsors"]["type"][sponsor_type].append(sponsor["name"])

            # pulling sponsor party from dict
            sponsor_party = sponsorPartyDict.get(sponsor["party_id"])
            if sponsor_party is not None:
                bills_json[bill_id]["agg"]["sponsors"]["sponsor-party"][sponsor_party] += 1

        # Aggregating sponsor type into string for each type
        for sponsorType in bills_json[bill_id]["agg"]["sponsors"]["type"]:
            bills_json[bill_id]["agg"]["sponsors"]["type"][sponsorType] = ", ".join(bills_json[bill_id]["agg"]["sponsors"]["type"][sponsorType])

        # Aggregating sponsor parties into string for each type
        bills_json[bill_id]["agg"]["sponsors"]["sponsor-party"] = ", ".join(f"{party_name}: {count}" for party_name, count in bills_json[bill_id]["agg"]["sponsors"]["sponsor-party"].items() if count > 0)

    # Saving Populated bills.json 
    with open("stateData/" + state + "/data/bills.json", "w") as file:
        json.dump(bills_json, file, indent=5)
        
print("Parsing & Aggregating Datasets Complete")

# Exporting Aggregated Data to Excel Workbook
The code below will export the aggregated data into a single Excel Workbook which contains individual sheets that represent each state in *stateCodes*.

In [0]:
%pip install openpyxl
# Creating Excel file sheet to populate with bills by state
with pd.ExcelWriter("US_Legislation_Collected " + datetime.datetime.now().strftime("%m-%d-%Y") + ".xlsx") as writer:
    for state in stateCodes:
        # Reading in aggregated data for state
        with open("stateData/" + state + "/data/bills.json", "r") as file:
            bills_json = json.load(file)
        # Excel File Column Names
        col = [
            "latest_update",
            "relevance",
            "state",
            "bill number",
            "date introduced",
            "description",
            "status date",
            "status label",
            "latest action",
            "state link", 
            "legiscan link",
            "chamber", 
            "party",
            "sponsor parties",
            "primary sponsor",
            "co-sponsor",
            "joint sponsor", 
            "generic/unspecified sponsor",
        ]

        df = pd.DataFrame(columns=col)
        for bill_id in bills_json:
            # Skipping over bills that weren't in the dataset
            if "state" not in bills_json[bill_id]["agg"]: continue
            
            newRow = {
                "latest_update": bills_json[bill_id]["agg"]["latest_update"],
                "relevance": bills_json[bill_id]["agg"]["relevance"],
                "state": bills_json[bill_id]["agg"]["state"],
                "bill number": bills_json[bill_id]["agg"]["bill number"],
                "date introduced": bills_json[bill_id]["agg"]["date introduced"],
                "description": bills_json[bill_id]["agg"]["description"],
                "status date": bills_json[bill_id]["agg"]["status"]["date"],
                "status label": bills_json[bill_id]["agg"]["status"]["label"],
                "latest action": bills_json[bill_id]["agg"]["status"]["action"],
                "state link": bills_json[bill_id]["agg"]["state link"],
                "legiscan link": bills_json[bill_id]["agg"]["legiscan link"],
                "chamber": bills_json[bill_id]["agg"]["chamber"],
                "party": bills_json[bill_id]["agg"]["party"],
                "sponsor parties": bills_json[bill_id]["agg"]["sponsors"]["sponsor-party"],
                "primary sponsor": bills_json[bill_id]["agg"]["sponsors"]["type"]["primary"],
                "co-sponsor": bills_json[bill_id]["agg"]["sponsors"]["type"]["co"],
                "joint sponsor": bills_json[bill_id]["agg"]["sponsors"]["type"]["joint"],
                "generic/unspecified sponsor": bills_json[bill_id]["agg"]["sponsors"]["type"]["generic-unspecified"]
                }
            df.loc[len(df)] = newRow

        df.to_excel(writer, sheet_name=state, index=False)
        print("Excel Sheet Completed For: " + state)
print("Excel Workbook with all states Complete")
        